# Parsing train data from auto.ru for car price prediction

## Import

In [1]:
# import
import time
from collections.abc import Iterable
from collections.abc import Sequence

import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.webdriver import WebDriver
import asyncio
import aiohttp
import json
import re

import pprint as pp
from IPython.display import display

print('Import is completed.')

Import is completed.


## Functions

In [102]:
# noinspection PyBroadException
def parse_car_data(car_url:str, content, features_to_extract:list):
    """Parse data from car ad page content trying to be as (reasonably) close to test set as possible."""

    car_data = {}

    car_page = BeautifulSoup(content, 'html.parser')

    # 'bodyType'
    try: car_data['bodyType'] = car_page.find('li', class_='CardInfoRow CardInfoRow_bodytype').find('a', class_='Link Link_color_black').string
    except: pass

    # get info from script
    try:
        # 'brand', 'color', 'description',  'productionDate' and 'priceCurrency'
        ld_json_scripts = car_page.find_all('script', type="application/ld+json")
        for script in ld_json_scripts:
            cur_json = json.loads(script.string)
            if cur_json['@type'] == 'https://schema.org/Product':
                car_data.update({k:cur_json.get(k,None) for k in ['brand', 'color', 'description', 'productionDate']})
                car_data['priceCurrency'] = cur_json['offers']['priceCurrency']
                break
    except:
        pass

    # 'car_url'
    car_data['car_url'] = car_url

    # engine info
    try:
        engine_texts = car_page.find('li', class_='CardInfoRow CardInfoRow_engine').find('div').text.split()

        # 'engineDisplacement'
        try: car_data['engineDisplacement'] = engine_texts[0]
        except: pass

        # 'enginePower'
        try: car_data['enginePower'] = engine_texts[3]
        except: pass

        # 'fuelType'
        try: car_data['fuelType'] = engine_texts[-1]
        except: pass
    except:
        pass

    # get info from state script
    try:
        json_script_state = json.loads(car_page.find('script', type="application/json", id='initial-state').string)

        # 'complectation_dict'
        try: car_data['complectation_dict'] = str(json_script_state['card']['vehicle_info']['complectation'])
        except: pass

        # 'equipment_dict'
        try: car_data['equipment_dict'] = str(json_script_state['card']['vehicle_info']['equipment'])
        except: pass

        # 'super_gen'
        try: car_data['super_gen'] = str(json_script_state['card']['vehicle_info']['tech_param'])
        except: pass

        # 'modelDate'
        try: car_data['modelDate'] = json_script_state['card']['vehicle_info']['super_gen']['year_from']
        except: pass

        # 'model_info'
        try: car_data['model_info'] = str(json_script_state['card']['vehicle_info']['model_info'])
        except: pass

        # 'model_name' (no sense but let's just add to be more like test set)
        try: car_data['model_name'] = json_script_state['card']['vehicle_info']['model_info']['code']
        except: pass

        # 'name' (no sense but let's just add to be more like test set)
        try: car_data['name'] = json_script_state['card']['vehicle_info']['tech_param']['human_name']
        except: pass

        # 'numberOfDoors'
        try: car_data['numberOfDoors'] = json_script_state['card']['vehicle_info']['configuration']['doors_count']
        except: pass
    except:
        pass

    # 'image'
    try: car_data['image'] = 'https:' + car_page.select('link[as=image]')[0]['href']
    except: pass

    # 'mileage'
    try: car_data['mileage'] = ''.join(re.findall('\d',car_page.find('li', class_='CardInfoRow CardInfoRow_kmAge').find_all('span')[1].string))
    except: pass

    # 'parsing_unixtime'
    car_data['parsing_unixtime'] = int(time.time())

    # 'vehicleTransmission',
    try: car_data['vehicleTransmission'] = car_page.find('li', class_='CardInfoRow CardInfoRow_transmission').find_all('span')[1].string
    except: pass

    # 'Владельцы'
    try: car_data['Владельцы'] = car_page.find('li', class_='CardInfoRow CardInfoRow_ownersCount').find_all('span')[1].text.replace(u'\xa0', u' ')
    except: pass

    # 'Владение'
    try: car_data['Владение'] = car_page.find('li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[1].text
    except: pass

    # 'ПТС'
    try: car_data['ПТС'] = car_page.find('li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[1].text
    except: pass

    # 'Привод'
    try: car_data['Привод'] = car_page.find('li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[1].text
    except: pass

    # 'Руль'
    try: car_data['Руль'] = car_page.find('li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[1].text
    except: pass

    # 'Состояние'
    try: car_data['Состояние'] = car_page.find('li', class_='CardInfoRow CardInfoRow_state').find_all('span')[1].text
    except: pass

    # 'Таможня'
    try: car_data['Таможня'] = car_page.find('li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[1].text
    except: pass

    # 'sell_id'
    # no need for training

    # 'vehicleConfiguration'
    # all this information is presented in other features - no need for train

    # 'vendor'
    # not found such info on site, but we can easily infer it later from test set

    # add price column
    try: car_data['Price'] = ''.join(re.findall('\d', car_page.find('span', class_='OfferPriceCaption__price').text))
    except: pass

    # fill not extracted features with None and rearrange output dict keys to follow test set format
    output_dict = {}
    for feature in features_to_extract+['Price']:
        if feature in car_data:
            output_dict[feature] = car_data[feature]
        else:
            output_dict[feature] = None

    return output_dict

def init_webdriver():
    """Webdriver initialization function. Configure webdriver to ensure correct car ads parsing."""

    # disable geolocation
    geoDisabled = webdriver.FirefoxOptions()
    geoDisabled.set_preference("geo.enabled", False)
    geoDisabled.set_preference("geo.provider.use_corelocation", False)
    geoDisabled.set_preference("geo.prompt.testing", False)
    geoDisabled.set_preference("geo.prompt.testing.allow", False)

    return webdriver.Firefox(options=geoDisabled)

def parse_car_urls(brand:str, driver:WebDriver):
    """Collect all car ad urls for specific brand via specific driver."""

    brand_url = f'https://auto.ru/cars/{brand}/all/'
    driver.get(brand_url)
    brand_page = BeautifulSoup(driver.page_source, 'html.parser')

    # get number of pages for brand
    num_pages = int(brand_page.find('span', class_='ControlGroup ControlGroup_responsive_no ControlGroup_size_s ListingPagination__pages') \
                                .find_all('a')[-1].text)
    car_urls = []
    for page_idx in range(1,num_pages+1):
        print(f'Brand \'{brand}\' (page {page_idx} from {num_pages})', end='\r', flush=True)
        page_url = brand_url + f'?page={page_idx}'
        driver.get(page_url)
        cur_page = BeautifulSoup(driver.page_source, 'html.parser')
        car_links = cur_page.find_all('a', class_='Link ListingItemTitle__link')
        car_urls.extend([link['href'] for link in car_links])

        time.sleep(1)

    print('')

    return car_urls

In [3]:
# load given test data
DATA_DIR = 'Data'
df_test = pd.read_csv(f'../{DATA_DIR}/test.csv')
display(df_test.head(5))
df_test.info()
df_test.describe()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

,mileage,modelDate,numberOfDoors,parsing_unixtime,productionDate,sell_id
count,34686.000000,34686.000000,34686.000000,3.468600e+04,34686.000000,3.468600e+04
mean,162009.767889,2007.074728,4.450816,1.603287e+09,2009.264602,1.098300e+09
std,100676.559489,7.415894,0.703040,1.493071e+05,7.047661,1.911225e+07
min,1.000000,1904.000000,0.000000,1.603107e+09,1904.000000,2.665000e+03
25%,91153.500000,2004.000000,4.000000,1.603221e+09,2006.000000,1.099049e+09
50%,149779.500000,2008.000000,5.000000,1.603254e+09,2011.000000,1.100911e+09
75%,215000.000000,2012.000000,5.000000,1.603290e+09,2014.000000,1.101245e+09
max,1000000.000000,2020.000000,5.000000,1.603710e+09,2020.000000,1.101375e+09


In [4]:
# get html page for navigation
url = 'https://auto.ru'
response = requests.get(url)
print('GET url status=', response.status_code)
page = BeautifulSoup(response.text,'html.parser')
print(page.prettify()[:500], '.....')

GET url status= 200
<!DOCTYPE html>
<html data-reactroot="" lang="ru">
 <head>
  <link as="script" href="https://auto.ru/_crpd/2W5I8yt76/b2530f-ah6D/GgBdzIYYcCBRKMb4AePeu15H21gDkDZ4uvt444bcqkZoSu3X1jye0fP9ptkU_Lo5gMNeGZtzrJp19-pl9aVQB1uTzczE3XsqgWjC9RgmXPREglbaBlWuLmyu7viXzeqBogZuxpe1md4u_28cBjc98p8NmlyR-ikZ8vFq738mkkUbGMfdSZk-aNWvxbzepts9iQCb0wPTJ-lj5Cb03MnqCSWBIc9QctdXYrgvF809efYRy15VF3KkWn7_5-cx6VrLlI" nonce="53tRKdiuhAG010lXQYee+w==" rel="preload"/>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv .....


In [5]:
# get all brands presented on site
brand_links = page.find_all('a',class_='IndexMarks__item')
brand_urls = list(map(lambda a: a['href'], brand_links))
pp.pprint(brand_urls)
site_brands = [u.split('/')[-3] for u in brand_urls]
site_brands

['https://auto.ru/cars/vaz/all/',
 'https://auto.ru/cars/audi/all/',
 'https://auto.ru/cars/bmw/all/',
 'https://auto.ru/cars/chery/all/',
 'https://auto.ru/cars/chevrolet/all/',
 'https://auto.ru/cars/citroen/all/',
 'https://auto.ru/cars/daewoo/all/',
 'https://auto.ru/cars/ford/all/',
 'https://auto.ru/cars/geely/all/',
 'https://auto.ru/cars/haval/all/',
 'https://auto.ru/cars/honda/all/',
 'https://auto.ru/cars/hyundai/all/',
 'https://auto.ru/cars/infiniti/all/',
 'https://auto.ru/cars/kia/all/',
 'https://auto.ru/cars/land_rover/all/',
 'https://auto.ru/cars/lexus/all/',
 'https://auto.ru/cars/mini/all/',
 'https://auto.ru/cars/mazda/all/',
 'https://auto.ru/cars/mercedes/all/',
 'https://auto.ru/cars/mitsubishi/all/',
 'https://auto.ru/cars/nissan/all/',
 'https://auto.ru/cars/opel/all/',
 'https://auto.ru/cars/peugeot/all/',
 'https://auto.ru/cars/porsche/all/',
 'https://auto.ru/cars/renault/all/',
 'https://auto.ru/cars/skoda/all/',
 'https://auto.ru/cars/ssang_yong/all/',
 

['vaz',
 'audi',
 'bmw',
 'chery',
 'chevrolet',
 'citroen',
 'daewoo',
 'ford',
 'geely',
 'haval',
 'honda',
 'hyundai',
 'infiniti',
 'kia',
 'land_rover',
 'lexus',
 'mini',
 'mazda',
 'mercedes',
 'mitsubishi',
 'nissan',
 'opel',
 'peugeot',
 'porsche',
 'renault',
 'skoda',
 'ssang_yong',
 'subaru',
 'suzuki',
 'toyota',
 'volkswagen',
 'volvo',
 'gaz',
 'uaz']

In [6]:
# brands in test data
brands = [brand.lower() for brand in df_test['brand'].unique()]
brands

['skoda',
 'audi',
 'honda',
 'volvo',
 'bmw',
 'nissan',
 'infiniti',
 'mercedes',
 'toyota',
 'lexus',
 'volkswagen',
 'mitsubishi']

In [7]:
# check if we have all needed brands on site
brands <= site_brands

True

Great, we have all brands from test set on site.
So, get ad urls from site for all brands that are in test set.

In [ ]:
# prepare webdriver to bypass site parsing protection
driver = init_webdriver()

try:
    driver.get('https://auto.ru/')
    time.sleep(1)

    # reject geolocation request in order to parse page by page (it is now allowed when you don't do it first..)
    no_location_button = driver.find_element_by_xpath('//span[@data-decision="no"]')
    no_location_button.click()
    time.sleep(3)

    # collect car urls for brands using already configured webdriver
    urls = []
    for brand in brands:
        print(f"Processing brand \'{brand}\':")
        urls.extend(parse_car_urls(brand, driver))

    path = f'Data/car_urls__{time.strftime("%H_%M__%d_%m_%Y")}.csv'
    pd.DataFrame(data=urls, columns=['car_url']).to_csv(path, index=False)
except Exception as e:
    print(e)
finally:
    driver.quit()

In [9]:
# let's see what features are in test set
test_features = list(df_test.columns)
pp.pprint(test_features)

['bodyType',
 'brand',
 'car_url',
 'color',
 'complectation_dict',
 'description',
 'engineDisplacement',
 'enginePower',
 'equipment_dict',
 'fuelType',
 'image',
 'mileage',
 'modelDate',
 'model_info',
 'model_name',
 'name',
 'numberOfDoors',
 'parsing_unixtime',
 'priceCurrency',
 'productionDate',
 'sell_id',
 'super_gen',
 'vehicleConfiguration',
 'vehicleTransmission',
 'vendor',
 'Владельцы',
 'Владение',
 'ПТС',
 'Привод',
 'Руль',
 'Состояние',
 'Таможня']


Let's parse data from collected car ads urls.

In [10]:
# load the most recent car ads urls
car_urls = pd.read_csv(DATA_DIR+'/car_urls__03_01__31_01_2022.csv')['car_url']
car_urls.shape

(42305,)

In [60]:
class Batcher:
    def __init__(self, src_list:Sequence, batch_size:int):
        self.src_list = src_list
        self.batch_size = batch_size
        self.batch_count = len(src_list) // batch_size
        self.idx_from = None
        self.idx_to = None

    # noinspection PyRedundantParentheses
    def batches(self):
        for i in range(self.batch_count):
            self.idx_from, self.idx_to = i*self.batch_size, (i+1)*self.batch_size
            yield (i, self.src_list[self.idx_from : self.idx_to])
        if self.batch_size * self.batch_count < len(self.src_list):
            yield (self.batch_count, self.src_list[self.batch_size * self.batch_count:])

async def get_car_data_async(task_idx, car_url, session, features_to_extract):
    try:
        # print(f'Task {task_idx} started.')
        # print(f'get_car_data_async(): get url {car_url}')
        async with session.get(url=car_url) as response:
            content = await response.read()

            # print(f'Task {task_idx} completed.')
            output_dict = parse_car_data(car_url, content.decode('utf-8'), features_to_extract)

    except Exception as e:
        print(f"Unable to get url {car_url} due to {e.__class__} with args={e.args}.")
        # create empty dict
        output_dict = dict.fromkeys(features_to_extract)

    return output_dict

async def process_batch_async(urls, features_to_extract):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[get_car_data_async(i, url, session, features_to_extract) for i, url in enumerate(urls)])
        return ret

In [103]:
batcher = Batcher(car_urls[:10], batch_size=5)

path = f'{DATA_DIR}/parsed_car_data__{time.strftime("%H_%M__%d_%m_%Y")}.csv'

for batch_idx, batch_urls in batcher.batches():
    print(f"Processing batch {batch_idx+1} from {batcher.batch_count}. Urls range [{batcher.idx_from} to {batcher.idx_to}] from total {car_urls.shape[0]} urls")
    beg_time = time.time()
    batch_result = await process_batch_async(batch_urls, test_features)
    print(f'Batch processing time={time.time() - beg_time}')
    df_batch = pd.DataFrame(batch_result)
    df_batch.index += batch_idx * batcher.batch_size

    # append batch data to csv file
    with open(path,'a+', encoding='utf-8') as file:
        file.write(df_batch.to_csv(header=True if batch_idx == 0 else False, index_label='№'))

    time.sleep(2)

print('Car parsing is completed.')

Processing batch 1 from 2. Urls range [0 to 5] from total 42305 urls
Batch processing time=19.832295179367065
Processing batch 2 from 2. Urls range [5 to 10] from total 42305 urls
Batch processing time=2.6998751163482666
Car parsing is completed.


Check the amount of useful records in parsed data:

In [104]:
df_parsed = pd.read_csv('Data/parsed_car_data__17_55__02_02_2022.csv')
df_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42305 entries, 0 to 42304
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   №                     42305 non-null  int64  
 1   bodyType              33195 non-null  object 
 2   brand                 41635 non-null  object 
 3   car_url               42305 non-null  object 
 4   color                 41635 non-null  object 
 5   complectation_dict    38510 non-null  object 
 6   description           41635 non-null  object 
 7   engineDisplacement    32624 non-null  float64
 8   enginePower           32624 non-null  float64
 9   equipment_dict        41552 non-null  object 
 10  fuelType              32624 non-null  object 
 11  image                 30612 non-null  object 
 12  mileage               33195 non-null  float64
 13  modelDate             40061 non-null  float64
 14  model_info            41552 non-null  object 
 15  model_name         

In [105]:
df_parsed[~df_parsed['bodyType'].isna() & ~df_parsed['Price'].isna()].shape

(30426, 34)

So we have ~30_000 (potentially) useful records for training. It is comparable with test set size (~35_000) - not bad and there is a hope.

## Trials

In [86]:
resp = requests.get('https://auto.ru/cars/new/group/skoda/karoq/22535699/22687243/1106682611-cf692f2d/')
car_page = BeautifulSoup(resp.content.decode('utf-8'), 'html.parser')

''.join(re.findall('\d', car_page.find('span', class_='OfferPriceCaption__price').text))

'2655000'